In [71]:
###Logistic Regression & RForests

from collections import Counter
import pandas as pd

#Handle Imbalance Data
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

#ML ALgorithims
from matplotlib import pyplot
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier, plot_importance
from sklearn.neural_network import MLPClassifier

In [2]:
#reading our no null master file
df = pd.read_csv('df_master.csv', index_col=0)

/Users/leotsang/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Reading our data

train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

In [4]:
train.columns

Index(['player_name', 'school', 'conference', 'GP', 'Min_per', 'ORtg', 'usg',
       'eFG', 'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM',
       'FTA', 'FT_per', '2PM', '2PA', '2P_per', '3PM', '3PA', '3P_per',
       'blk_per', 'stl_per', 'ftr', 'yr', 'ht', 'num', 'porpag', 'adjoe',
       'pfr', 'year', 'pid', 'type', 'rec-rk', 'ast/tov', 'rimmade',
       'rimmade + rimmiss', 'midmade', 'midmade + midmiss',
       'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+mismiss)', 'dunksmade',
       'dunksmiss + dunksmade', 'dunksmade/(dunksmade+dunksmiss)', 'pick',
       'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm',
       'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk',
       'pts', 'pos', '2PA_pg', '2PM_pg', '3PA_pg', '3PM_pg', 'yr_cat', 'ht_in',
       'drafted'],
      dtype='object')

In [5]:
#stats to look at

per_stats = ['Min_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FT_per', '2P_per', '3P_per', 
             'blk_per', 'stl_per', 'GP', 'ht_in', 'yr_cat']
box_stats = ['GP', 'mp', 'pts', '2PA_pg', '2PM_pg', '3PA_pg', '3PM_pg','oreb', 'dreb', 'treb',
             'ast', 'stl', 'blk', 'ht_in', 'yr_cat', 'FTA', 'FTM', 'ftr']
adv_stats = ['GP', 'mp', 'pts', '2PA_pg', '2PM_pg', '3PA_pg', '3PM_pg', 'oreb', 'dreb', 'treb',
             'ast', 'stl', 'blk', 'ht_in', 'yr_cat', 'bpm', 'obpm', 'dbpm', 'FTA', 'FTM', 'ftr']

per_adv = ['Min_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FT_per', '2P_per', '3P_per', 
             'blk_per', 'stl_per', 'GP', 'ht_in', 'yr_cat', 'bpm', 'obpm', 'dbpm']

all_stats = ['GP', 'Min_per', 'ORtg', 'usg',
       'eFG', 'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM',
       'FTA', 'FT_per', '2PM', '2PA', '2P_per', '3PM', '3PA', '3P_per',
       'blk_per', 'stl_per', 'ftr', 'porpag', 'adjoe',
       'pfr','year', 'rec-rk', 'ast/tov', 
       'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm',
       'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk',
       'pts', '2PA_pg', '2PM_pg', '3PA_pg', '3PM_pg', 'yr_cat', 'ht_in']

In [6]:
#delete all missing values
train = train.dropna(axis=0, subset=per_stats)
train = train.dropna(axis=0, subset=box_stats)
train = train.dropna(axis=0, subset=adv_stats)

#delete mssing value in test set as well

test = test.dropna(axis=0, subset=per_stats)
test = test.dropna(axis=0, subset=box_stats)
test = test.dropna(axis=0, subset=adv_stats)

In [7]:
#set our 2 test-set aside
test_2018 = test[test['year']==2018]
test_2019 = test[test['year']==2019]

In [ ]:
##SMOTE

over = SMOTE(sampling_strategy=0.025)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('over', over), ('under', under),('model', model)]
pipeline = Pipeline(steps=steps)
X, y = pipeline.fit_resample(X,y)

Counter(X), Counter(y)


In [58]:
#our preliminary model: Log Reg
tests = [per_stats, box_stats, adv_stats, per_adv] #all_stats]

for i in tests:
    
    model = LogisticRegression(max_iter=10000)
    model.fit(train[i], train['drafted'])
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    accuracy = accuracy_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('accuracy: {} recall: {} precision: {} predictions: {}'.format(accuracy, recall, precision, sum(predictions)))


0.9889666878845745 0.0392156862745098 0.4 5.0
0.987481434330575 0.2549019607843137 0.38235294117647056 34.0
0.9883301506471462 0.17647058823529413 0.4090909090909091 22.0
0.9906641205177169 0.21568627450980393 0.7333333333333333 15.0


In [68]:
#preliminary Random Forest

tests = [per_stats, box_stats, adv_stats, per_adv] #all_stats]

for i in tests:
    
    model = RandomForestClassifier()
    model.fit(train[i], train['drafted'])
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    accuracy = accuracy_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('accuracy: {} recall: {} precision: {} predictions: {}'.format(accuracy, recall, precision, sum(predictions)))



accuracy: 0.988542329726289 recall: 0.0 precision: 0.0 predictions: 3.0
accuracy: 0.9887545088054318 recall: 0.0784313725490196 precision: 0.4 predictions: 10.0
accuracy: 0.9896032251220029 recall: 0.1568627450980392 precision: 0.5714285714285714 predictions: 14.0
accuracy: 0.9889666878845745 recall: 0.09803921568627451 precision: 0.45454545454545453 predictions: 11.0


In [73]:
#preliminary  Adaboost

tests = [per_stats, box_stats, adv_stats, per_adv] #all_stats]

for i in tests:
    
    model = AdaBoostClassifier()
    model.fit(train[i], train['drafted'])
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    accuracy = accuracy_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('accuracy: {} recall: {} precision: {} predictions: {}'.format(accuracy, recall, precision, sum(predictions)))


accuracy: 0.9866327180140039 recall: 0.19607843137254902 precision: 0.3125 predictions: 32.0
accuracy: 0.9879057924888606 recall: 0.27450980392156865 precision: 0.4117647058823529 predictions: 34.0
accuracy: 0.9902397623594313 recall: 0.39215686274509803 precision: 0.5714285714285714 predictions: 35.0
accuracy: 0.9887545088054318 recall: 0.19607843137254902 precision: 0.45454545454545453 predictions: 22.0


In [69]:
###preliminary MLP

tests = [per_stats, box_stats, adv_stats, per_adv] #all_stats]

for i in tests:
    
    model = MLPClassifier()
    model.fit(train[i], train['drafted'])
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    accuracy = accuracy_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('accuracy: {} recall: {} precision: {} predictions: {}'.format(accuracy, recall, precision, sum(predictions)))

accuracy: 0.9889666878845745 recall: 0.0392156862745098 precision: 0.4 predictions: 5.0
accuracy: 0.9798429874814343 recall: 0.5294117647058824 precision: 0.2755102040816326 predictions: 98.0
accuracy: 0.9898154042011458 recall: 0.0784313725490196 precision: 0.8 predictions: 5.0
accuracy: 0.9904519414385742 recall: 0.21568627450980393 precision: 0.6875 predictions: 16.0


In [60]:
#with Smote LogREG####

tests = [per_stats, box_stats, adv_stats, per_adv] #all_stats]

for i in tests:
    
    model = LogisticRegression(max_iter=10000)
    oversample = SMOTE(sampling_strategy=0.025)
    X, y = oversample.fit_resample(train[i], train['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    accuracy = accuracy_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('accuracy: {} recall: {} precision: {} predictions: {}'.format(accuracy, recall, precision, sum(predictions)))



accuracy: 0.9872692552514322 recall: 0.3333333333333333 precision: 0.3953488372093023 predictions: 43.0
accuracy: 0.9849352853808614 recall: 0.47058823529411764 precision: 0.35294117647058826 predictions: 68.0
accuracy: 0.986420538934861 recall: 0.49019607843137253 precision: 0.3968253968253968 predictions: 63.0
accuracy: 0.9887545088054318 recall: 0.43137254901960786 precision: 0.4782608695652174 predictions: 46.0


In [66]:
###WITH SMOTE XGBOOOST#####

tests = [per_stats, box_stats, adv_stats, per_adv] #all_stats]

for i in tests:
    
    model = XGBClassifier(max_depth=4)
    oversample = SMOTE(sampling_strategy=0.025)
    X, y = oversample.fit_resample(train[i], train['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    accuracy = accuracy_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('Accuracy: {} Recall: {} Precision: {} Pred: {}'.format(accuracy, recall, precision, sum(predictions)))

accuracy: 0.9862083598557182 recall: 0.21568627450980393 precision: 0.3055555555555556 predictions: 36.0
accuracy: 0.9870570761722894 recall: 0.29411764705882354 precision: 0.375 predictions: 40.0
accuracy: 0.9902397623594313 recall: 0.49019607843137253 precision: 0.5555555555555556 predictions: 45.0
accuracy: 0.9900275832802886 recall: 0.3333333333333333 precision: 0.5666666666666667 predictions: 30.0


In [70]:
###WITH SMOTE MLP#####

tests = [per_stats, box_stats, adv_stats, per_adv] #all_stats]

for i in tests:
    
    model = MLPClassifier()
    oversample = SMOTE(sampling_strategy=0.025)
    X, y = oversample.fit_resample(train[i], train['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[i])
    predictions = [round(value) for value in y_pred]

    accuracy = accuracy_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('Accuracy: {} Recall: {} Precision: {} Pred: {}'.format(accuracy, recall, precision, sum(predictions)))

Accuracy: 0.9870570761722894 Recall: 0.09803921568627451 Precision: 0.25 Pred: 20.0
Accuracy: 0.9900275832802886 Recall: 0.3333333333333333 Precision: 0.5666666666666667 Pred: 30.0
Accuracy: 0.9891788669637174 Recall: 0.47058823529411764 Precision: 0.5 Pred: 48.0
Accuracy: 0.9811160619562911 Recall: 0.5882352941176471 Precision: 0.30612244897959184 Pred: 98.0
